In [4]:
import warnings
warnings.filterwarnings("ignore")

#### Installing Necessary Libraries

In [5]:
!pip install pandas
!pip install networkx
!pip install matplotlib
!pip install plotly
!pip install scipy
!pip install vaderSentiment
!pip install seaborn
!pip install bertopic
!pip install python-louvain
!python -m spacy download en_core_web_sm --user
!pip install -U spacy

  Using cached python_dateutil-2.9.0.post0-py2.py3-none-any.whl.metadata (8.4 kB)
  Using cached pytz-2025.2-py2.py3-none-any.whl.metadata (22 kB)
  Using cached tzdata-2025.2-py2.py3-none-any.whl.metadata (1.4 kB)
  Using cached six-1.17.0-py2.py3-none-any.whl.metadata (1.7 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.7/10.7 MB 1.8 MB/s eta 0:00:00a 0:00:01m
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.1/5.1 MB 3.1 MB/s eta 0:00:00a 0:00:01
Using cached python_dateutil-2.9.0.post0-py2.py3-none-any.whl (229 kB)
Using cached pytz-2025.2-py2.py3-none-any.whl (509 kB)
Using cached tzdata-2025.2-py2.py3-none-any.whl (347 kB)
Using cached six-1.17.0-py2.py3-none-any.whl (11 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 1.6 MB/s eta 0:00:00a 0:00:01
  Using cached cycler-0.12.1-py3-none-any.whl.metadata (3.8 kB)
  Using cached pyparsing-3.2.3-py3-none-any.whl.metadata (5.0 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.1/8.1 MB 1.1 MB/s eta 0:00:0000:0100:010m
Using ca

#### Necessary Imports

In [6]:
import pandas as pd
import numpy as np
import json
import plotly.express as px
import networkx as nx
import community
import matplotlib.pyplot as plt
import community.community_louvain as community_louvain
import plotly.graph_objects as go
from community import community_louvain
import spacy
import re
import seaborn as sns
from collections import Counter
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.decomposition import LatentDirichletAllocation
from sklearn.metrics import silhouette_score
from umap import UMAP

# Load Spacy NLP model
nlp = spacy.load("en_core_web_sm")

ModuleNotFoundError: No module named 'plotly'

#### Loading and Preprocessing data

In [ ]:
with open("InvestmentClub_submissions_refined.json", "r", encoding="utf-8") as file:
    data1 = json.load(file)

with open("InvestmentClub_comments_refined.json", "r", encoding="utf-8") as file:
    data2 = json.load(file)

df_sub = pd.DataFrame(data1)
df_com= pd.DataFrame(data2)

df_com["unique_id"] = df_com["parent_id"].str.replace("t3_", "").str.replace("t1_", "")
df_com["created_utc"] = pd.to_datetime(df_com["created_utc"], unit="s")
df_sub["created_utc"] = pd.to_datetime(df_sub["created_utc"], unit="s")

In [ ]:
print("Length of Submissions Dataset",len(df_sub))
print("Length of Comments Dataset",len(df_com))

#### Custom Data Overview: Inspecting Reddit Submissions and Comments

In [ ]:
def custom_info(df):
    print(f"<class 'pandas.core.frame.DataFrame'>")
    print(f"RangeIndex: {df.shape[0]} entries, 0 to {df.shape[0] - 1}")
    print(f"Data columns (total {df.shape[1]} columns):")
    print(f"{' #':<4} {'Column':<35} {'Null Count':<20} {'Dtype'}")
    print("-" * 80)

    for i, col in enumerate(df.columns):
        non_null_count = df[col].isna().sum()
        dtype = df[col].dtype
        print(f"{i:<4} {col:<35} {non_null_count:<20} {dtype}")

    print(f"\ndtypes: {', '.join([f'{t}({list(df.dtypes).count(t)})' for t in df.dtypes.unique()])}")
    print(f"memory usage: {df.memory_usage(deep=True).sum() / (1024 ** 2):.2f} MB")

In [ ]:
custom_info(df_sub)

In [ ]:
df_com.info()

#### Merging Posts and Comments for Network Analysis

In [ ]:
# Columns to retain from Posts Dataset
posts_columns = [
    'id',
    'author',
    'title',
    'created_utc',
    'score',
    'num_comments',
]

# Columns to retain from Comments Dataset
comments_columns = [
    'id',
    'author',
    'unique_id',
    'parent_id',
    'body',
    'created_utc',
    'score',
    'link_id',
    'ups',
    'downs'
]

df_sub = df_sub[posts_columns]
df_com = df_com[comments_columns]

new_df = df_com[df_com['unique_id'].isin(df_sub['id'])]
merged_data = new_df.merge(df_sub, left_on='unique_id', right_on='id', how='inner', suffixes=('_comment', '_post'))

# Combine title and body columns into a single text column
merged_data['text'] = merged_data['title'] + ' ' + merged_data['body']

# merged_data = merged_data.sample(n=500, random_state=42)

In [ ]:
merged_data.columns

In [ ]:
merged_data.shape

In [ ]:
merged_data.info()

In [ ]:
# Get the top 5 most active users based on interactions
top_users = merged_data["author_comment"].value_counts().head(5)
top_5_users = top_users.index.tolist()

# Filter dataset for interactions involving the top 5 users
filtered_df = merged_data[merged_data['author_comment'].isin(top_5_users)]

In [ ]:
# Initialize directed graph
G_top_5_users = nx.DiGraph()

# Add edges from filtered dataset
for _, row in filtered_df.iterrows():
    parent = row['author_comment']
    child = row['unique_id']
    if pd.notna(parent) and pd.notna(child):
        G_top_5_users.add_edge(parent, child)

In [ ]:
# Calculate centrality metrics
degree_centrality = nx.degree_centrality(G_top_5_users)
betweenness_centrality = nx.betweenness_centrality(G_top_5_users)
closeness_centrality = nx.closeness_centrality(G_top_5_users)

def get_node_positions(graph):
    return nx.spring_layout(graph, seed=42, k=0.3)

# Get positions for top 5 users
pos_top_5_users = get_node_positions(G_top_5_users)

def extract_plot_data(graph, pos):
    node_x, node_y, node_labels = [], [], []
    for node, (x, y) in pos.items():
        node_x.append(x)
        node_y.append(y)
        node_labels.append(node)

    edge_x, edge_y = [], []
    for edge in graph.edges():
        x0, y0 = pos[edge[0]]
        x1, y1 = pos[edge[1]]
        edge_x.extend([x0, x1, None])
        edge_y.extend([y0, y1, None])

    return node_x, node_y, node_labels, edge_x, edge_y

# Extract node and edge positions
node_x, node_y, node_labels, edge_x, edge_y = extract_plot_data(G_top_5_users, pos_top_5_users)

# Scale node sizes dynamically
node_sizes = [degree_centrality[node] * 300 for node in G_top_5_users.nodes()]

# Normalize degree centrality values for coloring
min_dc, max_dc = min(degree_centrality.values()), max(degree_centrality.values())

def create_degree_centrality_plot(node_sizes):

    fig = go.Figure()
    fig.add_trace(go.Scatter(
        x=edge_x, y=edge_y,
        line=dict(width=0.5, color="gray"),
        hoverinfo="none",
        mode="lines"
    ))
    fig.add_trace(go.Scatter(
        x=node_x, y=node_y,
        mode="markers+text",
        marker=dict(
            size=node_sizes,
            color=[degree_centrality[node] for node in G_top_5_users.nodes()],
            colorscale="YlOrRd",
            colorbar=dict(title="Degree Centrality"),
            opacity=0.8
        ),
        text=node_labels,
        textposition="top center",
        hoverinfo="text"
    ))
    fig.update_layout(
        title="Degree Centrality of Top 5 Users",
        showlegend=False,
        hovermode="closest",
        margin=dict(b=0, l=0, r=0, t=40),
        xaxis=dict(showgrid=False, zeroline=False, showticklabels=False),
        yaxis=dict(showgrid=False, zeroline=False, showticklabels=False),
    )
    return fig

def create_btwn_closns_centrality_plot(title, centrality_values, color_scale):
    """Creates a Plotly graph for betweenness or closeness centrality with fixed node sizes."""
    fig = go.Figure()
    fig.add_trace(go.Scatter(
        x=edge_x, y=edge_y,
        line=dict(width=0.5, color="gray"),
        hoverinfo="none",
        mode="lines"
    ))
    fig.add_trace(go.Scatter(
        x=node_x, y=node_y,
        mode="markers+text",
        marker=dict(
            size=12,
            color=[centrality_values[node] for node in G_top_5_users.nodes()],
            colorscale=color_scale,
            colorbar=dict(title=title),
            opacity=0.8
        ),
        text=node_labels,
        textposition="top center",
        hoverinfo="text"
    ))
    fig.update_layout(
        title=title,
        showlegend=False,
        hovermode="closest",
        margin=dict(b=0, l=0, r=0, t=40),
        xaxis=dict(showgrid=False, zeroline=False, showticklabels=False),
        yaxis=dict(showgrid=False, zeroline=False, showticklabels=False),
    )
    return fig

### Closeness Centrality for the Top 5 Most Active Users

Closeness Centrality for these top 5 users means that they have the shortest average distance to all other users in the subreddit community. They are positioned in the network in such a way that they can interact with or "reach" other users quickly (i.e., fewer steps are needed to connect to other users)

In [ ]:
# Create plots for each centrality measure
fig_closeness = create_btwn_closns_centrality_plot("Closeness Centrality of Top 5 Users", closeness_centrality, "Blues")

# Show plots
fig_closeness.show()

### Betweeness Centrality Analysis of the Top 5 Users

Betweenness Centrality for these top 5 users measures how often they act as bridges between other users in the subreddit community. These users facilitate interactions by connecting different parts of the network, meaning they play a crucial role in information flow and community cohesion.

In [ ]:
# Create plots for each centrality measure
fig_betweenness = create_btwn_closns_centrality_plot("Betweenness Centrality of Top 5 Users", betweenness_centrality, "Blues")

# Show plots
fig_betweenness.show()

### Degree Centrality

Degree Centrality for these top 5 users reflects the number of direct connections they have within the subreddit community. Users with higher degree centrality appear larger in the visualization, indicating their greater number of interactions. Additionally, their colors range from yellow to red, where deeper red hues correspond to higher degree centrality values. This visually highlights the most influential users in terms of direct engagement with others.

In [ ]:
# Create plots for each centrality measure
fig_degree = create_degree_centrality_plot(node_sizes)

# Show plots
fig_degree.show()

### Directed Interaction Graphs for the Top 5 Most Active Users

In [ ]:
for user in top_5_users:

    # Filter dataset for interactions involving this user
    user_df = merged_data[(merged_data["author_comment"] == user) | (merged_data["unique_id"] == user)]

    # Create a directed graph
    G_user = nx.DiGraph()

    # Add edges from the filtered dataset
    for _, row in user_df.iterrows():
        parent = row["author_comment"]
        child = row["unique_id"]
        if pd.notna(parent) and pd.notna(child):
            G_user.add_edge(parent, child)

    # Use a spring layout for better visualization
    pos_user = get_node_positions(G_user)


    # Extract node positions
    node_x, node_y, node_labels = [], [], []
    for node, (x, y) in pos_user.items():
        node_x.append(x)
        node_y.append(y)
        node_labels.append(node)

    # Extract edge positions
    edge_x, edge_y = [], []
    for edge in G_user.edges():
        x0, y0 = pos_user[edge[0]]
        x1, y1 = pos_user[edge[1]]
        edge_x.extend([x0, x1, None])
        edge_y.extend([y0, y1, None])

    # Create a directed graph visualization using Plotly
    fig_user = go.Figure()

    # Add edges (lines)
    fig_user.add_trace(go.Scatter(
        x=edge_x, y=edge_y,
        line=dict(width=0.5, color="gray"),
        hoverinfo="none",
        mode="lines"
    ))

    # Add nodes with labels
    fig_user.add_trace(go.Scatter(
        x=node_x, y=node_y,
        mode="markers+text",
        marker=dict(size=12, color=["red" if node == user else "blue" for node in node_labels], opacity=0.8),
        text=node_labels,
        textposition="top center",
        hoverinfo="text"
    ))


    fig_user.update_layout(
        title=f"Directed Interaction Graph for {user}",
        showlegend=False,
        hovermode="closest",
        margin=dict(b=0, l=0, r=0, t=40),
        xaxis=dict(showgrid=False, zeroline=False, showticklabels=False),
        yaxis=dict(showgrid=False, zeroline=False, showticklabels=False),
    )


    fig_user.show()

### Topic Modelling

In [ ]:
# Custom stopword list
with open("stopwordFile.txt", "r") as f:
    custom_stopwords = set(f.read().splitlines())

# Preprocess Text (remove stopwords, lemmatize, clean)
def preprocess(text):
    if pd.isnull(text):
        return ""
    doc = nlp(text.lower())
    tokens = [token.lemma_ for token in doc if token.lemma_ not in custom_stopwords
              and not token.is_stop and not token.is_punct]
    return " ".join(tokens)

merged_data['cleaned_text'] = merged_data['text'].fillna('')
merged_data["cleaned_text"] = merged_data["cleaned_text"].apply(preprocess)

In [ ]:
# TF-IDF Vectorization
vectorizer = TfidfVectorizer(max_features=5000, stop_words=list(custom_stopwords), ngram_range=(1, 2))
X_tfidf = vectorizer.fit_transform(merged_data["cleaned_text"])

# UMAP for dimensionality reduction
umap_model = UMAP(n_neighbors=20, n_components=10, min_dist=0.0, metric="cosine")
X_umap = umap_model.fit_transform(X_tfidf.toarray())

# LDA Topic Modeling
lda = LatentDirichletAllocation(n_components=10, random_state=42)
lda.fit(X_tfidf)

# Extracting top words for each topic
words = vectorizer.get_feature_names_out()
topics_info = []
for i, topic in enumerate(lda.components_):
    top_words_indices = topic.argsort()[-10:][::-1]
    top_words = [words[j] for j in top_words_indices]

    topic_label = f"Topic {i+1}"

    topic_name = ", ".join(top_words)

    # Create the count (frequency of each word in the topic)
    word_count = [topic[j] for j in top_words_indices]

    # Append the topic information to the list
    topics_info.append([topic_label, topic_name, word_count])

# Convert topics info into DataFrame
topics_df = pd.DataFrame(topics_info, columns=["topic_label", "topic_name", "count"])

# Assign topics to the merged_data DataFrame
topic_assignments = lda.transform(X_tfidf)
merged_data["topic"] = topic_assignments.argmax(axis=1)

# Count the number of rows assigned to each topic
topic_counts = merged_data['topic'].value_counts().reset_index()
topic_counts.columns = ['topic_label', 'row_count']

# Convert topic_label to string to match with topics_df
topic_counts['topic_label'] = topic_counts['topic_label'].apply(lambda x: f"Topic {x + 1}")

topics_df = topics_df.merge(topic_counts, on='topic_label', how='left')
merged_data['topic'] = merged_data['topic'].astype(str)
merged_data = merged_data.merge(topics_df[['topic_label', 'topic_name']], left_on='topic', right_on='topic_label', how='left')

sorted_topics_df = topics_df[["topic_label", "topic_name", "row_count"]].sort_values(by="row_count", ascending=False)
sorted_topics_df

### Top Topics by Frequency

In [ ]:
# Create interactive bar chart
fig = px.bar(
    sorted_topics_df,
    x="topic_label",
    y="row_count",
    text="row_count",
    hover_data=["topic_name"],
    labels={"topic_label": "Topic", "row_count": "Number of Mentions"},
    title="Topic Mentions in Dataset"
)

fig.update_traces(textposition="outside")
fig.update_layout(xaxis_tickangle=-45)

fig.show()

## Community Analysis

In [ ]:
# Function to build graph for each topic
def build_graph(df, topic):
    G = nx.Graph()
    topic_data = df[df["topic"] == topic]

    for _, row in topic_data.iterrows():
        user = row["author_comment"]
        reply_to = row["unique_id"]

        G.add_node(user)

        if pd.notna(reply_to):
            G.add_node(reply_to)
            G.add_edge(user, reply_to)

    return G

def plot_community_graph(G, topic):
    if G.number_of_nodes() == 0:
        print(f"Skipping {topic} - No interactions")
        return

    partition = community_louvain.best_partition(G)
    pos = nx.spring_layout(G, seed=42)
    node_colors = [partition[node] for node in G.nodes()]

    edge_x, edge_y = [], []
    for edge in G.edges():
        x0, y0 = pos[edge[0]]
        x1, y1 = pos[edge[1]]
        edge_x.extend([x0, x1, None])
        edge_y.extend([y0, y1, None])

    fig = go.Figure()

    fig.add_trace(go.Scatter(x=edge_x, y=edge_y, mode="lines", line=dict(width=0.5, color="gray"), hoverinfo="none"))
    node_x, node_y = zip(*[pos[node] for node in G.nodes()])
    node_text = list(G.nodes())

    fig.add_trace(go.Scatter(
        x=node_x, y=node_y, mode="markers",
        marker=dict(size=10, color=node_colors, colorscale="Viridis", showscale=True),
        text=node_text, hoverinfo="text"
    ))

    fig.update_layout(
        title=f"Community Graph for Topic: {topic}",
        showlegend=False, hovermode="closest",
        margin=dict(b=0, l=0, r=0, t=40),
        xaxis=dict(showgrid=False, zeroline=False, showticklabels=False),
        yaxis=dict(showgrid=False, zeroline=False, showticklabels=False)
    )
    fig.show()

# Generate Graphs for Each Topic
unique_topics = merged_data["topic"].dropna().unique()
for topic in unique_topics:
    G = build_graph(merged_data, topic)
    plot_community_graph(G, topic)

In [ ]:
# topics_df.to_csv("topics_names.csv")

### Community Talking about Warren Buffet

In [ ]:
# Filter relevant comments
rel_comments = merged_data["cleaned_text"].str.contains(r"\b(warren|buffet|warren buffet)\b", case=False, na=False)

# Count occurrences by year
merged_data["year"] = pd.to_datetime(merged_data["created_utc_comment"], unit="s").dt.year
com_count = merged_data[rel_comments].groupby("year").size().reset_index(name="count")

fig = px.line(com_count, x="year", y="count", markers=True, title="Mentions of Warren Buffet Over Time")
fig.show()

### Rich-Club Coefficient During Peak Years of Discussion

In [ ]:
# Function to build a graph
def build_graph(df):
    G = nx.Graph()
    for _, row in df.iterrows():
        user = row["author_comment"]
        reply_to = row["unique_id"]
        G.add_node(user)
        if pd.notna(reply_to):
            G.add_node(reply_to)
            G.add_edge(user, reply_to)
    return G

# Filter dataset for peak years
peak_years = [2019,2020,2021]
filtered_data = merged_data[(rel_comments) & (merged_data["year"].isin(peak_years))]

# Build the graph
G_warren = build_graph(filtered_data)

# Compute rich club coefficient
rich_club = nx.rich_club_coefficient(G_warren, normalized=False)

degrees = list(rich_club.keys())
coefficients = list(rich_club.values())

# Extend the degree range
extended_degrees = np.linspace(min(degrees), max(degrees) + 20, 100)
interpolated_coefficients = np.interp(extended_degrees, degrees, coefficients)

# Plot the rich club coefficient with extended degree scale
plt.figure(figsize=(10, 5))
plt.plot(extended_degrees, interpolated_coefficients, marker="o", linestyle="-", color="b", label="Rich Club Coefficient")
plt.xlim([0, max(degrees) + 10])
plt.ylim([0, max(coefficients) + 0.1])
plt.xlabel("Degree (k)")
plt.ylabel("Rich Club Coefficient φ(k)")
plt.title("Extended Rich Club Coefficient for Warren Buffet Related Discussions (2019-2021)")
plt.legend()
plt.grid(True)
plt.show()

## Sensitivity Analysis

In [ ]:
# Function to compute the size of the largest connected component
def largest_component_size(G):
    if len(G.nodes()) == 0:
        return 0
    largest_component = max(nx.connected_components(G), key=len)
    return len(largest_component) / len(G.nodes()) * 100

# Sensitivity analysis function for a specific topic
def sensitivity_analysis_for_topic(data, fraction_steps=10):
    G = nx.Graph()

    # Build the graph for the filtered topic data
    for _, row in filtered_data.iterrows():
        author = row['author_comment']
        parent = row['unique_id']

        if pd.notna(parent) and parent in filtered_data['unique_id'].values:
            G.add_edge(author, parent)

    print(f"Graph contains {len(G.nodes())} nodes and {len(G.edges())} edges")

    # If graph is empty, return empty DataFrame instead of tuple
    if len(G.nodes()) == 0:
        return pd.DataFrame(columns=["Fraction of Nodes Removed", "Largest Component Size (%)"])

    # Prepare for sensitivity analysis
    node_count = len(G.nodes())
    fractions = np.linspace(0, 1, fraction_steps)  # Fraction of nodes to remove
    largest_component_sizes = []

    # Sorted list of nodes based on degree (you can adjust this as needed)
    sorted_nodes = sorted(G.nodes(), key=lambda x: G.degree(x), reverse=True)

    for fraction in fractions:
        # Number of nodes to remove
        nodes_to_remove = int(fraction * node_count)

        # Remove nodes progressively based on the sorted list
        nodes_removed = sorted_nodes[:nodes_to_remove]
        G.remove_nodes_from(nodes_removed)

        # Compute the size of the largest connected component
        largest_size = largest_component_size(G)
        largest_component_sizes.append(largest_size)

    result_df = pd.DataFrame({
        "Fraction of Nodes Removed": fractions,
        "Largest Component Size (%)": largest_component_sizes
    })

    return result_df

# Perform sensitivity analysis on Topic 2
result_topic = sensitivity_analysis_for_topic(merged_data)

if not result_topic.empty:
    plt.figure(figsize=(10, 6))
    plt.plot(result_topic["Fraction of Nodes Removed"], result_topic["Largest Component Size (%)"], label="Topic 2", color="red")
    plt.xlabel("Fraction of Nodes Removed from the network")
    plt.ylabel("Size of Largest Component (%)")
    plt.title("Sensitivity Analysis")
    plt.legend()
    plt.grid(True)
    plt.show()
else:
    print("No data available for this topic")

In [ ]:
# rich_club_df = pd.DataFrame(list(rich_club.items()), columns=['Degree', 'Coefficient'])
# rich_club_df.to_csv("rich_club_coefficient.csv")

## Z-Score

In [ ]:
# Calculate Z-score for the 'count' of mentions per year
mean_count = com_count['count'].mean()  # Mean of counts
std_count = com_count['count'].std()  # Standard deviation of counts

# Compute the Z-score for each year
com_count['z_score'] = (com_count['count'] - mean_count) / std_count

# Create an interactive plot using Plotly
fig = go.Figure()

# Add traces for Z-scores vs. count of Warren Buffett mentions
fig.add_trace(go.Scatter(
    x=com_count['count'],
    y=com_count['z_score'],
    mode='markers+lines',
    marker=dict(size=12, color=com_count['z_score'], colorscale='Viridis', showscale=True),
    text=com_count['year'],
    hoverinfo='text+name',
))


fig.update_layout(
    title="Z-Score vs. Count of Topics Related to Warren Buffett",
    xaxis_title="Number of Mentions of Warren Buffett",
    yaxis_title="Z-Score",
    hovermode='closest',
    plot_bgcolor='rgba(0,0,0,0)',
    xaxis=dict(showgrid=True),
    yaxis=dict(showgrid=True),
)

fig.show()

##### The Z-scores reveal how public interest in Warren Buffett fluctuates over time. Years with negative Z-scores (e.g., -0.87) show when mentions were below average, likely during quieter periods in his career. In contrast, positive Z-scores, particularly the peak of 1.94, highlight years of exceptional interest, possibly due to significant events like major investments or public statements. The data suggests that Buffett’s media presence and influence are dynamic, with certain years standing out for higher visibility, while others show a dip in public engagement.

### Monthly Engagement Trends: All Users vs Super Users

In [ ]:
merged_data["month"] = merged_data["created_utc_comment"].dt.to_period("M").astype(str)

# Overall Monthly Engagement of All Users
monthly_total_all = merged_data.groupby("month").size().reset_index(name="total_comments")

fig_all = px.line(
    monthly_total_all, x="month", y="total_comments", markers=True,
    title="Overall Engagement Trends of All Users",
    labels={"month": "Month", "total_comments": "Total Comments"},
)

fig_all.update_layout(hovermode="x", xaxis_tickangle=-45)
fig_all.show()

# Monthly Engagement Trends for Super Users
super_user_data = merged_data[merged_data["author_comment"].isin(top_5_users)]

# Aggregate data by month for super users
monthly_total_super = super_user_data.groupby("month").size().reset_index(name="total_comments")

fig_super = px.line(
    monthly_total_super, x="month", y="total_comments", markers=True,
    title="Overall Engagement Trends of Super Users",
    labels={"month": "Month", "total_comments": "Total Comments"},
)

fig_super.update_layout(hovermode="x", xaxis_tickangle=-45)
fig_super.show()

In [ ]:
filter_dts=merged_data.copy()

# Filter the data to include only entries after 2020
filter_dts = filter_dts[filter_dts["created_utc_comment"] >= "2020-01-01"]

# Create a new column for the week (using "W-Mon" to make the week start on Monday)
filter_dts["week"] = filter_dts["created_utc_comment"].dt.to_period("W-Mon").astype(str)

# Filter only top 5 super users
super_user_data = filter_dts[filter_dts["author_comment"].isin(top_5_users)]

# Aggregate data by week for super users and all users
weekly_activity_all = filter_dts.groupby(["week", "author_comment"]).size().reset_index(name="comment_count")
weekly_activity_super = super_user_data.groupby(["week", "author_comment"]).size().reset_index(name="comment_count")

# Overall Engagement Trends - Total Comments Per Week for All Users
weekly_total_all = filter_dts.groupby("week").size().reset_index(name="total_comments")

fig2_all = px.line(
    weekly_total_all, x="week", y="total_comments", markers=True,
    title="Overall Engagement Trends of All Users (Weekly After 2020)",
    labels={"week": "Week", "total_comments": "Total Comments"},
)

fig2_all.update_layout(hovermode="x", xaxis_tickangle=-45)
fig2_all.show()

# Overall Engagement Trends - Total Comments Per Week for Super Users
weekly_total_super = super_user_data.groupby("week").size().reset_index(name="total_comments")

fig2_super = px.line(
    weekly_total_super, x="week", y="total_comments", markers=True,
    title="Overall Engagement Trends of Super Users (Weekly After 2020)",
    labels={"week": "Week", "total_comments": "Total Comments"},
)

fig2_super.update_layout(hovermode="x", xaxis_tickangle=-45)
fig2_super.show()

### Analyzing Comments During Specific Time Periods

In [ ]:
# Define the two time periods
period_1_start = "2021-01-26"
period_1_end = "2021-02-01"
period_2_start = "2022-01-04"
period_2_end = "2022-01-10"

# Filter for comments within the specified time periods
df_period_1 = merged_data[
    (merged_data["created_utc_comment"] >= period_1_start) &
    (merged_data["created_utc_comment"] <= period_1_end)
]

df_period_2 = merged_data[
    (merged_data["created_utc_comment"] >= period_2_start) &
    (merged_data["created_utc_comment"] <= period_2_end)
]

# Display the number of comments in each period
print(f"Number of comments in {period_1_start} - {period_1_end}: {df_period_1.shape[0]}")
print(f"Number of comments in {period_2_start} - {period_2_end}: {df_period_2.shape[0]}")

In [ ]:
# df_period_1.to_csv("df_period_1.csv")
# df_period_2.to_csv("df_period_2.csv")

In [ ]:
df_period_1

In [ ]:
df_period_2